In [1]:
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from scipy.optimize import curve_fit,Bounds
from scipy.integrate import simps
import zfit_physics as zphysics
import zfit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import mplhep
import uncertainties 
from uncertainties import ufloat
import tensorflow as tf

plt.style.use('belle2')
#%jsroot on


%run /afs/desy.de/user/j/jipatel/MT/MC_fit/Files_After_Acceptance/Signal_Background_thesis_final/Function.ipynb

2024-09-11 20:21:13.835336: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-11 20:21:13.882245: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-11 20:21:13.883160: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-11 20:21:15.534765: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warnin

## 200

In [ ]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppi0,file_bkg_mu_Bppi0, 'mu', '521', 'pi0', 'mumu', 4, 'rec521', 'weights', 200
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"],  'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"], 'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"],  'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"],  'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)


    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_200.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_200.csv'")


RuntimeError: Function minimum is not valid: <FMin algorithm='Migrad' edm=0.0004035606241402507 edm_goal=0.001 errordef=0.5 fval=-5201.762980773633 has_accurate_covar=False has_covariance=True has_made_posdef_covar=False has_parameters_at_limit=False has_posdef_covar=False has_reached_call_limit=True has_valid_parameters=False hesse_failed=True is_above_max_edm=False is_valid=False nfcn=699 ngrad=0 reduced_chi2=nan time=3.7307504564523697>

## 150

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppi0,file_bkg_mu_Bppi0, 'mu', '521', 'pi0', 'mumu', 4, 'rec521', 'weights', 150
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_150.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_150.csv'")


Running iteration 1/300...
150
5454


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.629                 305.409  -     32   +     33       False
background_yield_0.629             5298.25  -     77   +     78       False
frac_Mbc_bkg1_0.629               0.408618  -  0.017   +  0.016       False
frac_deltaE_bkg1_0.629            0.445883  -  0.044   +  0.043       False
True
Iteration 1 - Signal Yield: 305.4084950485474 ± 32.213223570916945
Running iteration 2/300...
150
5454
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3483                  308.18  -     32   +     33       False
background_yield_0.3483              5296.1  -     78   +     78       False
frac_Mbc_bkg1_0.3483               0.407669  -  0.017   +  0.017       False
frac_deltaE_bkg1_0.3483            0.447524  -  0.0

## 100

In [4]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppi0,file_bkg_mu_Bppi0, 'mu', '521', 'pi0', 'mumu', 4, 'rec521', 'weights', 100
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_100.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_100.csv'")


Running iteration 1/300...
100
5454
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.693                 248.909  -     30   +     31       False
background_yield_0.693             5305.08  -     77   +     78       False
frac_Mbc_bkg1_0.693               0.410077  -  0.017   +  0.017       False
frac_deltaE_bkg1_0.693            0.446114  -  0.044   +  0.043       False
True
Iteration 1 - Signal Yield: 248.9087760724542 ± 30.886474271029687
Running iteration 2/300...
100
5454
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9712                 246.944  -     31   +     31       False
background_yield_0.9712             5307.05  -     77   +     78       False
frac_Mbc_bkg1_0.9712               0.410591  -  0.017   +  0.017       False
frac_deltaE_bkg

RuntimeError: Function minimum is not valid: <FMin algorithm='Migrad' edm=0.00046339169646177883 edm_goal=0.001 errordef=0.5 fval=-5004.223156067427 has_accurate_covar=False has_covariance=True has_made_posdef_covar=False has_parameters_at_limit=False has_posdef_covar=False has_reached_call_limit=True has_valid_parameters=False hesse_failed=True is_above_max_edm=False is_valid=False nfcn=824 ngrad=0 reduced_chi2=nan time=3.9754356150515378>

## 75

In [ ]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppi0,file_bkg_mu_Bppi0, 'mu', '521', 'pi0', 'mumu', 4, 'rec521', 'weights', 75
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_75.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_75.csv'")


## 50

In [ ]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppi0,file_bkg_mu_Bppi0, 'mu', '521', 'pi0', 'mumu', 4, 'rec521', 'weights', 50
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_50.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_50.csv'")


## 25

In [ ]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppi0,file_bkg_mu_Bppi0, 'mu', '521', 'pi0', 'mumu', 4, 'rec521', 'weights', 25
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_25.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_25.csv'")


## 10

In [ ]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppi0,file_bkg_mu_Bppi0, 'mu', '521', 'pi0', 'mumu', 4, 'rec521', 'weights', 10
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_10.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_10.csv'")


## 5

In [ ]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppi0,file_bkg_mu_Bppi0, 'mu', '521', 'pi0', 'mumu', 4, 'rec521', 'weights', 5
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_5.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_5.csv'")


## 2

In [ ]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppi0,file_bkg_mu_Bppi0, 'mu', '521', 'pi0', 'mumu', 4, 'rec521', 'weights', 2
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_2.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_2.csv'")


## 0

In [ ]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppi0,file_bkg_mu_Bppi0, 'mu', '521', 'pi0', 'mumu', 4, 'rec521', 'weights', 0
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_0.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg4_decay_mumu_signal_yields_and_errors_0.csv'")
